In [ ]:
# Install required packages
!pip install nx-arangodb
!pip install --upgrade langchain langchain-community langchain-openai langgraph
!pip install pandas matplotlib networkx
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which

In [ ]:
# Import the required modules
import networkx as nx
import nx_arangodb as nxadb
from arango import ArangoClient

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import randint
import re
import os
import random

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_community.graphs import ArangoGraph
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
from langchain_core.tools import tool

[06:26:46 +0000] [INFO]: NetworkX-cuGraph is available.
INFO:nx_arangodb:NetworkX-cuGraph is available.


In [ ]:
!nvidia-smi
!nvcc --version

Mon Mar 10 06:26:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install nx-cugraph-cu12 --extra-index-url https://pypi.nvidia.com # Requires CUDA-capable GPU

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [ ]:
# Set OpenAI API key directly
import os
os.environ["OPENAI_API_KEY"] = "xxx" # use ur api key

In [ ]:
# Function to process LinkedIn connections file
def process_connections_file(file_path, owner_name, id_offset=0):
    # Load the connections data
    connections_df = pd.read_csv(file_path)

    # Create a unique identifier for each connection
    connections_df['id'] = connections_df.index + id_offset
    connections_df['full_name'] = connections_df['First Name'] + ' ' + connections_df['Last Name']
    connections_df['owner'] = owner_name  # Add owner information

    # Select relevant columns - adjust these based on your actual CSV structure
    clean_df = connections_df[['id', 'full_name', 'Company', 'Position', 'Email Address', 'owner']]

    # Handle missing values
    clean_df = clean_df.fillna({'Company': 'Unknown', 'Position': 'Unknown', 'Email Address': ''})

    return clean_df

# Function to generate random skills based on job titles
def generate_random_skills(df):
    # Define skill pools by industry/role
    tech_skills = [
        "Python", "JavaScript", "SQL", "Java", "C++", "AWS", "Azure",
        "Docker", "Kubernetes", "Machine Learning", "Data Science",
        "React", "Node.js", "Cloud Computing", "DevOps", "Cybersecurity",
        "Artificial Intelligence", "Blockchain", "Big Data", "Data Analysis"
    ]

    business_skills = [
        "Project Management", "Strategic Planning", "Business Development",
        "Marketing", "Sales", "Leadership", "Negotiation", "Public Speaking",
        "Financial Analysis", "Market Research", "Customer Relationship Management",
        "Team Management", "Business Strategy", "Consulting", "Operations Management"
    ]

    creative_skills = [
        "Graphic Design", "UI/UX Design", "Content Creation", "Copywriting",
        "Video Production", "Photography", "Adobe Creative Suite", "Branding",
        "Social Media Marketing", "Content Strategy", "Animation", "Illustration"
    ]

    general_skills = [
        "Communication", "Teamwork", "Problem Solving", "Critical Thinking",
        "Time Management", "Adaptability", "Creativity", "Emotional Intelligence",
        "Attention to Detail", "Organization", "Research", "Presentation Skills"
    ]

    # Create a copy of the dataframe
    df_with_skills = df.copy()

    # Add skills based on position
    df_with_skills['skills'] = df_with_skills.apply(
        lambda row: assign_skills_by_position(
            row['Position'],
            tech_skills,
            business_skills,
            creative_skills,
            general_skills
        ),
        axis=1
    )

    return df_with_skills

def assign_skills_by_position(position, tech_skills, business_skills, creative_skills, general_skills):
    position = str(position).lower()

    # Determine which skill pools to use based on position keywords
    if any(keyword in position for keyword in ['engineer', 'developer', 'data', 'tech', 'software', 'it', 'analyst']):
        primary_pool = tech_skills
        secondary_pool = general_skills
        tertiary_pool = business_skills
    elif any(keyword in position for keyword in ['manager', 'director', 'executive', 'ceo', 'cfo', 'coo', 'founder', 'president']):
        primary_pool = business_skills
        secondary_pool = general_skills
        tertiary_pool = tech_skills
    elif any(keyword in position for keyword in ['design', 'creative', 'artist', 'writer', 'content', 'marketing']):
        primary_pool = creative_skills
        secondary_pool = general_skills
        tertiary_pool = business_skills
    else:
        primary_pool = general_skills
        secondary_pool = random.choice([tech_skills, business_skills, creative_skills])
        tertiary_pool = random.choice([tech_skills, business_skills, creative_skills])

    # Select 2-3 skills from primary pool
    num_primary = random.randint(2, 3)
    primary_skills = random.sample(primary_pool, min(num_primary, len(primary_pool)))

    # Select 1-2 skills from secondary pool
    num_secondary = random.randint(1, 2)
    secondary_skills = random.sample(secondary_pool, min(num_secondary, len(secondary_pool)))

    # Occasionally add a skill from tertiary pool
    tertiary_skills = []
    if random.random() < 0.3:  # 30% chance
        tertiary_skills = [random.choice(tertiary_pool)]

    # Combine skills and return as comma-separated string
    all_skills = primary_skills + secondary_skills + tertiary_skills
    return ", ".join(all_skills)

# Upload multiple LinkedIn connections CSV files
from google.colab import files
print("Upload Soham's LinkedIn connections CSV file:")
uploaded_soham = files.upload()  # This will prompt you to upload Soham's connections.csv file

print("\nUpload Emily's LinkedIn connections CSV file:")
uploaded_emily = files.upload()  # This will prompt you to upload Emily's connections.csv file

# Process each file
soham_df = process_connections_file(list(uploaded_soham.keys())[0], "Soham", id_offset=0)
emily_df = process_connections_file(list(uploaded_emily.keys())[0], "Emily", id_offset=10000)

# Combine the dataframes
combined_df = pd.concat([soham_df, emily_df], ignore_index=True)

# Generate random skills for connections
combined_df = generate_random_skills(combined_df)

# Display basic statistics
print("\nCombined LinkedIn Connections Data:")
print(f"Total connections from Soham: {len(soham_df)}")
print(f"Total connections from Emily: {len(emily_df)}")
print(f"Total combined connections: {len(combined_df)}")

# Display sample of combined data with skills
print("\nSample of combined data with generated skills:")
print(combined_df[['full_name', 'Company', 'Position', 'skills']].head())

Upload Soham's LinkedIn connections CSV file:


Saving sohamconnections.csv to sohamconnections (1).csv

Upload Emily's LinkedIn connections CSV file:


Saving emilyconnections.csv to emilyconnections (1).csv

Combined LinkedIn Connections Data:
Total connections from Soham: 494
Total connections from Emily: 470
Total combined connections: 964

Sample of combined data with generated skills:
           full_name                                     Company  \
0      Akash Sonowal                                NoBroker.com   
1       Sahil Lalani                             Stealth Startup   
2          Aman Daga                                      PayPal   
3  Ella Bullock-Papa  Stanford King Center on Global Development   
4     Jamie Burnette                                 Capital One   

                                            Position  \
0                                  Data Scientist II   
1                                  Software Engineer   
2          Tech Product Manager - AI Personalization   
3                                 Research Assistant   
4  Business Analyst - Portfolio Strategy & Analyt...   

             

In [ ]:
# Create a NetworkX graph with improved structure and FEWER edges
G = nx.Graph()

# Add nodes (connections) with attributes including skills
for idx, row in combined_df.iterrows():
    G.add_node(row['id'],
               name=row['full_name'],
               company=row['Company'],
               position=row['Position'],
               email=row['Email Address'],
               owner=row['owner'],
               skills=row['skills'])  # Add skills attribute

# Create a more selective edge creation strategy
# 1. For each person, connect to at most 5 random people from the same company
companies = combined_df['Company'].unique()
for company in companies:
    if company != 'Unknown':
        same_company = combined_df[combined_df['Company'] == company]['id'].tolist()
        # Only create edges if there are multiple people at the company
        if len(same_company) > 1:
            # For each person, connect to at most 5 others from same company
            for person in same_company:
                # Get other people at same company
                others = [p for p in same_company if p != person]
                # Connect to at most 5 random people (or fewer if not enough)
                connect_to = min(5, len(others))
                if connect_to > 0:
                    random_others = random.sample(others, connect_to)
                    for other in random_others:
                        G.add_edge(person, other, relationship='same_company')

# 2. Create edges between people with similar skills
# Extract all skills
all_skills = set()
for _, row in combined_df.iterrows():
    skills = row['skills'].lower().split(',')
    for skill in skills:
        all_skills.add(skill.strip())

# Create a dictionary mapping skills to people
skill_to_people = {skill: [] for skill in all_skills}
for idx, row in combined_df.iterrows():
    person_skills = [s.strip().lower() for s in row['skills'].split(',')]
    for skill in person_skills:
        if skill in skill_to_people:
            skill_to_people[skill].append(row['id'])

# Create edges between people with common skills (max 3 connections per skill)
for skill, people in skill_to_people.items():
    if len(people) > 1:
        for person in people:
            others = [p for p in people if p != person]
            # Connect to at most 3 random people with the same skill
            connect_to = min(3, len(others))
            if connect_to > 0:
                random_others = random.sample(others, connect_to)
                for other in random_others:
                    if not G.has_edge(person, other):
                        G.add_edge(person, other, relationship='similar_skills')

# 3. Create a limited number of edges between connections of the same owner
# Instead of connecting everyone to everyone, create a more realistic network
for owner in ["Soham", "Emily"]:
    owner_connections = combined_df[combined_df['owner'] == owner]['id'].tolist()
    # Create a more realistic network structure - each person connects to ~5 others
    for person in owner_connections:
        others = [p for p in owner_connections if p != person]
        # Connect to at most 5 random people (or fewer if not enough)
        connect_to = min(5, len(others))
        if connect_to > 0:
            random_others = random.sample(others, connect_to)
            for other in random_others:
                if not G.has_edge(person, other):
                    G.add_edge(person, other, relationship='same_owner')

# Basic graph statistics
print(f"Number of nodes (connections): {G.number_of_nodes()}")
print(f"Number of edges (relationships): {G.number_of_edges()}")

Number of nodes (connections): 964
Number of edges (relationships): 17881


In [ ]:
# Connect to ArangoDB using the provided credentials
client = ArangoClient(hosts="https://df7687e8971e.arangodb.cloud:8529")
db = client.db(username="root", password="1ADDF5Q0eJypkH0QdJNs", verify=True)

# Create collections if they don't exist
if not db.has_collection('LinkedIn_node'):
    db.create_collection('LinkedIn_node')
if not db.has_collection('LinkedIn_node_to_LinkedIn_node'):
    db.create_collection('LinkedIn_node_to_LinkedIn_node', edge=True)

# Create the graph in ArangoDB
if not db.has_graph('LinkedIn'):
    graph = db.create_graph('LinkedIn', edge_definitions=[
        {
            'edge_collection': 'LinkedIn_node_to_LinkedIn_node',
            'from_vertex_collections': ['LinkedIn_node'],
            'to_vertex_collections': ['LinkedIn_node']
        }
    ])
else:
    graph = db.graph('LinkedIn')

In [ ]:
# Clear existing data to avoid conflicts
try:
    db.collection('LinkedIn_node_to_LinkedIn_node').truncate()
    db.collection('LinkedIn_node').truncate()
    print("Cleared existing data in ArangoDB")
except Exception as e:
    print(f"Error clearing data: {e}")

# Add nodes to ArangoDB with error handling
node_count = 0
for node_id in G.nodes():
    node_data = G.nodes[node_id]
    # Create a document key from the node ID
    key = str(node_id)

    try:
        # Add the node to ArangoDB
        db.collection('LinkedIn_node').insert({
            '_key': key,
            'name': node_data.get('name', ''),
            'company': node_data.get('company', ''),
            'position': node_data.get('position', ''),
            'email': node_data.get('email', ''),
            'owner': node_data.get('owner', ''),
            'skills': node_data.get('skills', '')
        })
        node_count += 1

        # Print progress every 100 nodes
        if node_count % 100 == 0:
            print(f"Added {node_count} nodes so far...")
    except Exception as e:
        print(f"Error inserting node {key}: {e}")

# Add edges to ArangoDB with error handling
edge_count = 0
for u, v, data in G.edges(data=True):
    # Create edge document
    edge = {
        '_from': f'LinkedIn_node/{u}',
        '_to': f'LinkedIn_node/{v}',
        'relationship': data.get('relationship', 'connection')
    }

    try:
        # Add the edge to ArangoDB
        db.collection('LinkedIn_node_to_LinkedIn_node').insert(edge)
        edge_count += 1

        # Print progress every 100 edges
        if edge_count % 100 == 0:
            print(f"Added {edge_count} edges so far...")
    except Exception as e:
        # Skip duplicate edges
        pass

print(f"Graph successfully persisted to ArangoDB with {node_count} nodes and {edge_count} edges!")

Cleared existing data in ArangoDB
Added 100 nodes so far...
Added 200 nodes so far...
Added 300 nodes so far...
Added 400 nodes so far...
Error inserting node 426: [HTTP 400][ERR 600] VPackError error: Expecting digit
Added 500 nodes so far...
Added 600 nodes so far...
Error inserting node 10137: [HTTP 400][ERR 600] VPackError error: Expecting digit
Added 700 nodes so far...
Added 800 nodes so far...
Added 900 nodes so far...
Added 100 edges so far...
Added 200 edges so far...
Added 300 edges so far...
Added 400 edges so far...
Added 500 edges so far...
Added 600 edges so far...
Added 700 edges so far...
Added 800 edges so far...
Added 900 edges so far...
Added 1000 edges so far...
Added 1100 edges so far...
Added 1200 edges so far...
Added 1300 edges so far...
Added 1400 edges so far...
Added 1500 edges so far...
Added 1600 edges so far...
Added 1700 edges so far...
Added 1800 edges so far...
Added 1900 edges so far...
Added 2000 edges so far...
Added 2100 edges so far...
Added 2200 e

KeyboardInterrupt: 

In [ ]:
# Load the graph from ArangoDB for querying
G_adb = nx.Graph()

# Get all nodes from ArangoDB
cursor = db.aql.execute(
    "FOR doc IN LinkedIn_node RETURN doc"
)
for node in cursor:
    G_adb.add_node(
        node['_key'],
        name=node.get('name', ''),
        company=node.get('company', ''),
        position=node.get('position', ''),
        email=node.get('email', ''),
        owner=node.get('owner', ''),
        skills=node.get('skills', '')
    )

# Get all edges from ArangoDB
cursor = db.aql.execute(
    "FOR edge IN LinkedIn_node_to_LinkedIn_node RETURN edge"
)
for edge in cursor:
    from_key = edge['_from'].split('/')[1]
    to_key = edge['_to'].split('/')[1]
    G_adb.add_edge(from_key, to_key, relationship=edge.get('relationship', 'connection'))

print(f"Loaded graph from ArangoDB with {G_adb.number_of_nodes()} nodes and {G_adb.number_of_edges()} edges")

In [ ]:
# Create ArangoGraph object for LangChain
from langchain_community.graphs import ArangoGraph
arango_graph = ArangoGraph(db)

# Initialize the LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [ ]:
# Define tools for the agent
from langchain_core.tools import tool
from langchain_community.chains.graph_qa.arangodb import ArangoGraphQAChain
import re
import networkx as nx

@tool
def favourite_fruit(query: str):
    """You are responsible for responding to being asked what your favourite fruit is.
    You must say Avocado!
    """
    return "Avocado!"

@tool
def text_to_aql_to_text(query: str):
    """This tool is available to invoke the
    ArangoGraphQAChain object, which enables you to
    translate a Natural Language Query into AQL, execute
    the query, and translate the result back into Natural Language.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    chain = ArangoGraphQAChain.from_llm(
        llm=llm,
        graph=arango_graph,
        verbose=True,
        allow_dangerous_requests=True
    )

    result = chain.invoke(query)

    return str(result["result"])

@tool
def text_to_nx_algorithm_to_text(query):
    """This tool is available to invoke a NetworkX Algorithm on
    the ArangoDB Graph. You are responsible for accepting the
    Natural Language Query, establishing which algorithm needs to
    be executed, executing the algorithm, and translating the results back
    to Natural Language, with respect to the original query.

    If the query (e.g traversals, shortest path, etc.) can be solved using the Arango Query Language, then do not use
    this tool.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    print("1) Generating NetworkX code")

    text_to_nx = llm.invoke(f"""
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

    I have the following graph analysis query: {query}.

    Generate the Python Code required to answer the query using the `G_adb` object.

    Be very precise on the NetworkX algorithm you select to answer this query. Think step by step.
    Use PageRank algorithm when appropriate for influence or importance analysis.

    Only assume that networkx is installed, and other base python dependencies.

    Always store the PageRank scores in a variable called `pagerank_scores` before using them.
    Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

    Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

    Make sure that `FINAL_RESULT` stores a short & concise answer. Avoid setting this variable to a long sequence.

    Your code:
    """).content

    text_to_nx_cleaned = re.sub(r"^```python\n|```$", "", text_to_nx, flags=re.MULTILINE).strip()

    print("2) Executing NetworkX code")

    global_vars = {"G_adb": G_adb, "nx": nx}
    local_vars = {}

    try:
        exec(text_to_nx_cleaned, global_vars, local_vars)
        result = local_vars.get("FINAL_RESULT", "No result produced")
    except Exception as e:
        print(f"EXEC ERROR: {e}")
        return f"Error in NetworkX analysis: {e}"

    print("3) Generating natural language response")

    text_to_nl = llm.invoke(f"""
    I was asked: "{query}"

    I executed a NetworkX algorithm and got the following result: {result}

    Please translate this result into a natural language response that directly answers the original query.
    Be concise but informative. Explain any technical terms if necessary.
    """).content

    return text_to_nl

@tool
def hybrid_query_execution(query):
    """This tool is available to execute hybrid queries that combine AQL and NetworkX algorithms.
    It's designed for complex queries that require both data retrieval and graph analytics.
    """

    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    # Step 1: Plan the hybrid query approach
    print("Hybrid Query Planning")

    aql_planning = llm.invoke(f"""
    I need to answer this query about a LinkedIn network: "{query}"

    This requires a hybrid approach using both AQL for data retrieval and NetworkX for graph analytics.

    First, I need to determine what data to retrieve with AQL. Write a description of what
    data we should retrieve with AQL (don't write the actual AQL yet).

    Your response:
    """).content

    print("Hybrid Query Planning - AQL Component:")
    print(aql_planning)

    # Step 2: Generate and execute the AQL query
    aql_chain = ArangoGraphQAChain.from_llm(
        llm=llm,
        graph=arango_graph,
        verbose=False,
        allow_dangerous_requests=True
    )

    aql_result = aql_chain.invoke(f"Retrieve the following data: {aql_planning}")

    print("\nAQL Result:")
    print(aql_result["result"])

    # Step 3: Generate NetworkX code to analyze the retrieved data
    nx_code = llm.invoke(f"""
    I have a NetworkX Graph called `G_adb`. It has the following schema: {arango_graph.schema}

    I have retrieved the following data using AQL: {aql_result["result"]}

    Now I need to perform advanced analysis on this data to answer the original query: "{query}"

    Generate Python code using NetworkX algorithms to analyze this data and answer the query.
    Use PageRank algorithm when appropriate for influence or importance analysis.

    Always store the PageRank scores in a variable called `pagerank_scores` before using them.
    Always set the last variable as `FINAL_RESULT`, which represents the answer to the original query.

    Only provide python code that I can directly execute via `exec()`. Do not provide any instructions.

    Your code:
    """).content

    nx_code_cleaned = re.sub(r"^```python\n|```$", "", nx_code, flags=re.MULTILINE).strip()

    print("\nNetworkX Analysis Code:")
    print(nx_code_cleaned)

    # Step 4: Execute the NetworkX code
    global_vars = {"G_adb": G_adb, "nx": nx, "aql_data": aql_result["result"]}
    local_vars = {}

    try:
        exec(nx_code_cleaned, global_vars, local_vars)
        nx_result = local_vars.get("FINAL_RESULT", "No result produced")
    except Exception as e:
        print(f"EXEC ERROR: {e}")
        return f"Error in NetworkX analysis: {e}"

    # Step 5: Generate the final response
    final_response = llm.invoke(f"""
    I was asked: "{query}"

    First, I retrieved data using AQL: {aql_result["result"]}

    Then, I analyzed this data using NetworkX algorithms and got: {nx_result}

    Based on both the data retrieval and analysis, provide a comprehensive answer to the original query.
    Make sure to explain insights from both the graph structure and the analytics performed.

    Your response:
    """).content

    return final_response

In [ ]:
# Automatic Query Router
def determine_query_type(query):
    """
    Uses LLM to determine the best query type for a given question
    """
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

    response = llm.invoke(f"""
    I need to determine the best query type for this question about a LinkedIn network: "{query}"

    Please classify this query into one of these categories:

    1. SIMPLE QUERY: Basic retrieval questions that can be answered with AQL, like "Who works at Google?" or "Find all software engineers"

    2. COMPLEX QUERY: Questions requiring graph algorithms like PageRank, centrality measures, community detection, like "Who is the most influential person?" or "What communities exist in the network?"

    3. HYBRID QUERY: Questions requiring both data retrieval and complex analysis, like "Who are the most influential people at Microsoft?" or "Which second-degree connections should I prioritize meeting?"

    Respond with ONLY ONE of these exact phrases: "SIMPLE QUERY", "COMPLEX QUERY", or "HYBRID QUERY".
    """).content

    # Clean up response to get just the classification
    response = response.strip().upper()

    if "SIMPLE QUERY" in response:
        return "Simple Query", text_to_aql_to_text
    elif "COMPLEX QUERY" in response:
        return "Complex Query", text_to_nx_algorithm_to_text
    elif "HYBRID QUERY" in response:
        return "Hybrid Query", hybrid_query_execution
    else:
        return "Simple Query", text_to_aql_to_text  # Default to simple query

In [ ]:
# Test the agent with a few queries
print("Query 1: Who in my network works at Google?")
result1, type1 = query_graph("Who in my network works at Google?")
print(f"Query Type: {type1}")
print(result1)

print("\nQuery 2: Who has the most connections in my network?")
result2, type2 = query_graph("Who has the most connections in my network?")
print(f"Query Type: {type2}")
print(result2)

print("\nQuery 3: Find people in my network who have skills in data science")
result3, type3 = query_graph("Find people in my network who have skills in data science")
print(f"Query Type: {type3}")
print(result3)

print("\nQuery 4: Who are the most influential people at Microsoft in my network?")
result4, type4 = query_graph("Who are the most influential people at Microsoft in my network?")
print(f"Query Type: {type4}")
print(result4)

In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
import io
from PIL import Image

# Function to get list of companies for dropdown
def get_company_list():
    companies = set()
    for _, attr in G_adb.nodes(data=True):
        company = attr.get('company', '')
        if company and company != 'Unknown':
            companies.add(company)

    return ["All Companies"] + sorted(list(companies))

# Function to get list of skills for dropdown
def get_skills_list():
    all_skills = set()
    for _, attr in G_adb.nodes(data=True):
        skills = attr.get('skills', '').split(',')
        for skill in skills:
            skill = skill.strip()
            if skill:
                all_skills.add(skill)

    return ["All Skills"] + sorted(list(all_skills))

# Function to create a network visualization
def create_network_visualization(owner_filter=None, company_filter=None, skill_filter=None, limit=100):
    # Filter graph based on parameters
    filtered_nodes = set()

    # Apply owner filter
    if owner_filter and owner_filter != "All":
        filtered_nodes = {n for n, attr in G_adb.nodes(data=True) if attr.get('owner') == owner_filter}
    else:
        filtered_nodes = set(G_adb.nodes())

    # Apply company filter
    if company_filter and company_filter != "All Companies":
        company_nodes = {n for n, attr in G_adb.nodes(data=True) if attr.get('company') == company_filter}
        filtered_nodes = filtered_nodes.intersection(company_nodes)

    # Apply skill filter
    if skill_filter and skill_filter != "All Skills":
                skill_nodes = {n for n, attr in G_adb.nodes(data=True)
                      if skill_filter.lower() in attr.get('skills', '').lower()}
                        skill_nodes = {n for n, attr in G_adb.nodes(data=True)
                      if skill_filter.lower() in attr.get('skills', '').lower()}
        filtered_nodes = filtered_nodes.intersection(skill_nodes)

    # Create subgraph
    filtered_graph = G_adb.subgraph(filtered_nodes)

    # If graph is too large, take a sample
    if filtered_graph.number_of_nodes() > limit:
        # Get top nodes by degree centrality
        centrality = nx.degree_centrality(filtered_graph)
        top_nodes = sorted(centrality, key=centrality.get, reverse=True)[:limit]
        filtered_graph = filtered_graph.subgraph(top_nodes)

    # Create visualization
    plt.figure(figsize=(10, 8))

    # Use a more efficient layout algorithm
    pos = nx.spring_layout(filtered_graph, k=0.3, iterations=50, seed=42)

    # Color nodes by owner and relationship type
    node_colors = []
    for node in filtered_graph.nodes():
        owner = filtered_graph.nodes[node].get('owner', '')
        if owner == "Soham":
            node_colors.append("#4285F4")  # Blue for Soham
        elif owner == "Emily":
            node_colors.append("#EA4335")  # Red for Emily
        else:
            node_colors.append("#FBBC05")  # Yellow for unknown

    # Draw nodes with simpler styling
    nx.draw_networkx_nodes(filtered_graph, pos, node_size=100, node_color=node_colors, alpha=0.8)

    # Draw edges with colors based on relationship type
    same_company_edges = [(u, v) for u, v, d in filtered_graph.edges(data=True)
                          if d.get('relationship') == 'same_company']
    similar_skills_edges = [(u, v) for u, v, d in filtered_graph.edges(data=True)
                           if d.get('relationship') == 'similar_skills']
    same_owner_edges = [(u, v) for u, v, d in filtered_graph.edges(data=True)
                        if d.get('relationship') == 'same_owner']

    nx.draw_networkx_edges(filtered_graph, pos, edgelist=same_company_edges,
                          alpha=0.5, width=0.5, edge_color='green')
    nx.draw_networkx_edges(filtered_graph, pos, edgelist=similar_skills_edges,
                          alpha=0.5, width=0.5, edge_color='purple')
    nx.draw_networkx_edges(filtered_graph, pos, edgelist=same_owner_edges,
                          alpha=0.5, width=0.5, edge_color='orange')

    # Draw labels for only a subset of nodes to avoid clutter
    if filtered_graph.number_of_nodes() <= 30:
        # If few nodes, label all of them
        labels = {node: filtered_graph.nodes[node].get('name', '') for node in filtered_graph.nodes()}
        nx.draw_networkx_labels(filtered_graph, pos, labels=labels, font_size=8)
    else:
        # If many nodes, label only the top 15 by degree centrality
        centrality = nx.degree_centrality(filtered_graph)
        top_nodes = sorted(centrality, key=centrality.get, reverse=True)[:15]
        labels = {node: filtered_graph.nodes[node].get('name', '') for node in top_nodes}
        nx.draw_networkx_labels(filtered_graph, pos, labels=labels, font_size=8)

    # Add a legend
    plt.figtext(0.01, 0.01, "Blue: Soham's connections, Red: Emily's connections", fontsize=8)
    plt.figtext(0.01, 0.03, "Green: Same company, Purple: Similar skills, Orange: Same owner", fontsize=8)

    # Set title based on filters
    title = "LinkedIn Network"
    if owner_filter and owner_filter != "All":
        title += f" - {owner_filter}'s Connections"
    if company_filter and company_filter != "All Companies":
        title += f" at {company_filter}"
    if skill_filter and skill_filter != "All Skills":
        title += f" with {skill_filter} skills"

    plt.title(title)
    plt.axis('off')

    # Save figure to a buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=300, bbox_inches='tight')
    buf.seek(0)
    plt.close()

    # Convert buffer to PIL image
    img = Image.open(buf)

    # Add network statistics as text
    stats_text = f"""
    Network Statistics:
    - Nodes: {filtered_graph.number_of_nodes()}
    - Edges: {filtered_graph.number_of_edges()}
    - Density: {nx.density(filtered_graph):.4f}
    - Avg. Clustering: {nx.average_clustering(filtered_graph):.4f}
    """

    return img, stats_text

# Function to get network statistics
def get_network_stats(owner_filter=None):
    # Filter graph by owner if specified
    if owner_filter and owner_filter != "All":
        owner_nodes = [n for n, attr in G_adb.nodes(data=True) if attr.get('owner') == owner_filter]
        filtered_graph = G_adb.subgraph(owner_nodes)
    else:
        filtered_graph = G_adb

    # Calculate PageRank
    pagerank_scores = nx.pagerank(filtered_graph)

    # Get top influential people
    top_influential = sorted(pagerank_scores.items(), key=lambda x: x[1], reverse=True)[:5]
    top_influential_names = [(filtered_graph.nodes[node].get('name', ''), score)
                            for node, score in top_influential]

    # Basic statistics
    stats = {
        "Total Connections": filtered_graph.number_of_nodes(),
        "Total Relationships": filtered_graph.number_of_edges(),
        "Network Density": nx.density(filtered_graph),
        "Average Clustering": nx.average_clustering(filtered_graph),
    }

    # Get top companies
    companies = {}
    for node in filtered_graph.nodes():
        company = filtered_graph.nodes[node].get('company', 'Unknown')
        if company in companies:
            companies[company] += 1
        else:
            companies[company] = 1

    top_companies = sorted(companies.items(), key=lambda x: x[1], reverse=True)[:5]

    # Get top skills
    skills_count = {}
    for node in filtered_graph.nodes():
        skills = filtered_graph.nodes[node].get('skills', '').split(',')
        for skill in skills:
            skill = skill.strip()
            if skill:
                if skill in skills_count:
                    skills_count[skill] += 1
                else:
                    skills_count[skill] = 1

    top_skills = sorted(skills_count.items(), key=lambda x: x[1], reverse=True)[:10]

    # Format the output as markdown
    output = f"""
    ## Network Statistics for {owner_filter if owner_filter != 'All' else 'All Connections'}

    ### Basic Metrics
    - **Total Connections**: {stats['Total Connections']}
    - **Total Relationships**: {stats['Total Relationships']}
    - **Network Density**: {stats['Network Density']:.4f}
    - **Average Clustering**: {stats['Average Clustering']:.4f}

    ### Top Influential People (PageRank)
    """

    for name, score in top_influential_names:
        output += f"- **{name}**: {score:.4f}\n"

    output += "\n### Top Companies\n"
    for company, count in top_companies:
        output += f"- **{company}**: {count} connections\n"

    output += "\n### Top Skills\n"
    for skill, count in top_skills:
        output += f"- **{skill}**: {count} connections\n"

    return output

# Create the Gradio interface for NetworkX AI
with gr.Blocks(title="NetworkX AI - LinkedIn Network Analyzer") as demo:
    gr.Markdown("""
    # NetworkX AI - LinkedIn Network Analyzer

    Analyze your professional network with AI-powered graph analytics. This tool helps you discover insights,
    find connections, and leverage your network more effectively.

    ## Features
    - Query your network using natural language
    - Visualize connections and relationships
    - Analyze network statistics and metrics
    - Discover skills and expertise across your network
    """)

    with gr.Tab("Query Network"):
        with gr.Row():
            with gr.Column(scale=3):
                query_input = gr.Textbox(
                    label="Ask a question about your network",
                    placeholder="e.g., Who are the most influential people in my network?",
                    lines=2
                )
                query_type = gr.Radio(
                    ["Auto Detect", "Simple Query", "Complex Query", "Hybrid Query"],
                    label="Query Type",
                    value="Auto Detect"
                )
                query_button = gr.Button("Submit Query")

            with gr.Column(scale=1):
                gr.Markdown("""
                ### Query Types
                - **Simple Query**: Basic retrieval (Who works at Google?)
                - **Complex Query**: Graph algorithms (Who is most influential?)
                - **Hybrid Query**: Combined analysis (Who should I meet next?)
                """)

        with gr.Row():
            query_result = gr.Textbox(label="Answer", lines=10)
            query_type_result = gr.Textbox(label="Query Type Used")

        query_button.click(
            lambda q, t: query_graph(q, None if t == "Auto Detect" else t),
            inputs=[query_input, query_type],
            outputs=[query_result, query_type_result]
        )

    with gr.Tab("Network Visualization"):
        with gr.Row():
            with gr.Column(scale=1):
                viz_owner_filter = gr.Radio(
                    ["All", "Soham", "Emily"],
                    label="Filter by network owner",
                    value="All"
                )
                viz_company_filter = gr.Dropdown(
                    choices=get_company_list(),
                    label="Filter by company",
                    value="All Companies"
                )
                viz_skill_filter = gr.Dropdown(
                    choices=get_skills_list(),
                    label="Filter by skill",
                    value="All Skills"
                )
                viz_button = gr.Button("Generate Visualization")

            with gr.Column(scale=3):
                viz_output = gr.Image(label="Network Visualization")
                viz_stats = gr.Textbox(label="Visualization Statistics", lines=5)

        viz_button.click(
            create_network_visualization,
            inputs=[viz_owner_filter, viz_company_filter, viz_skill_filter],
            outputs=[viz_output, viz_stats]
        )

    with gr.Tab("Network Statistics"):
        with gr.Row():
            stats_owner_filter = gr.Radio(
                ["All", "Soham", "Emily"],
                label="Filter by network owner",
                value="All"
            )
            stats_button = gr.Button("Generate Network Statistics")

        stats_output = gr.Markdown()

        stats_button.click(
            get_network_stats,
            inputs=stats_owner_filter,
            outputs=stats_output
        )

    with gr.Tab("Sample Queries"):
        gr.Markdown("""
        ## Sample Queries to Try

        ### Simple Queries
        - Who in my network works at Google?
        - Find all connections who are software engineers
        - Which companies have the most people in my network?
        - Who in my network has skills in Python?

        ### Complex Queries (Using PageRank and other algorithms)
        - Who has the most connections in my network?
        - Who are the most influential people in my network?
        - What is the most common job title in my network?
        - What skills clusters exist in my network?

        ### Hybrid Queries
        - Who are the most influential people at Microsoft in my network?
        - Find potential networking opportunities through second-degree connections
        - Which industry clusters exist in my network and who are the key people in each?
        - Who should I connect with to expand my network in the data science field?

        ### Comparison Queries
        - Compare Soham's and Emily's networks - who has more tech industry connections?
        - Who are the common connections between Soham and Emily?
        - Which network has more diverse companies represented?
        - Compare the skill distribution between Soham's and Emily's networks
        """)

    with gr.Tab("About NetworkX AI"):
        gr.Markdown("""
        ## About NetworkX AI

        NetworkX AI is a powerful tool for professional network analysis, developed to help professionals,
        recruiters, and business development teams leverage their LinkedIn connections more effectively.

        ### Key Benefits

        - **Discover Hidden Insights**: Uncover patterns and relationships in your network that aren't visible through standard LinkedIn interfaces
        - **Identify Key Influencers**: Find the most connected and influential people in your network
        - **Skill Mapping**: Understand the distribution of skills and expertise across your connections
        - **Strategic Networking**: Get recommendations on who to connect with next based on your goals

        ### Technology Stack

        - **Graph Database**: ArangoDB for efficient storage and querying of network relationships
        - **Graph Analytics**: NetworkX for advanced graph algorithms and metrics
        - **Natural Language Processing**: LangChain and OpenAI for conversational interface
        - **Visualization**: Matplotlib and NetworkX for interactive network visualization

        ### Privacy & Security

        Your LinkedIn data remains private and is only used for analysis within this application.
        No data is shared with third parties or used for any purpose other than providing insights to you.
        """)

# Launch the app
demo.launch(share=True)